In [41]:
import os
import pandas as pd
import requests
from dotenv import load_dotenv
import matplotlib.pyplot as plt
from tabulate import tabulate
import redshift_connector

In [42]:
load_dotenv('.env')

False

In [16]:
def extract_data1(symbol):
    try:
        url = f'https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY&symbol={symbol}&apikey={os.getenv("API_KEY")}'
        response = requests.get(url)
        json_data = response.json()
        data = pd.DataFrame(json_data['Monthly Time Series'])
        data = data.T
        data['symbol'] = symbol
        
        return data

    except requests.exceptions.RequestException as e:
        print(f"Error de solicitud: {e}")
        return None

In [17]:
data_nvidia = extract_data1('NVDA')
data_Cocacola = extract_data1('KO')
data_Visa = extract_data1('V')
data = pd.concat([data_nvidia , data_Cocacola, data_Visa], axis=0)

In [18]:
data.head()

,1. open,2. high,3. low,4. close,5. volume,symbol
2024-08-02,117.5300,120.1600,101.3700,107.2700,1005489790,NVDA
2024-07-31,123.4700,136.1500,102.5400,117.0200,6407092755,NVDA
2024-06-28,1136.2100,1255.8700,117.0100,123.5400,5234031077,NVDA
2024-05-31,850.7700,1158.1915,812.5456,1096.3300,964797076,NVDA
2024-04-30,902.9900,922.2500,756.0600,864.0200,1004973847,NVDA


In [19]:
def transform_data(data):
    data.rename(columns = {
        "1. open" : "open", 
        "2. high" : "high", 
        "3. low" : "low", 
        "4. close" : "close", 
        "5. volume" : "volume"
        }, inplace=True)
    
    for column in data[["open", "high", "low", "close", "volume"]]:
        data[column] = data[column].astype(float)

    data.reset_index(inplace=True)
    data.rename(columns={"index":"date"}, inplace=True)
    data.date = pd.to_datetime(data.date)

    return data

In [20]:
transformed_data = transform_data(data)
transformed_data.head()

,date,open,high,low,close,volume,symbol
0,2024-08-02,117.53,120.1600,101.3700,107.27,1.005490e+09,NVDA
1,2024-07-31,123.47,136.1500,102.5400,117.02,6.407093e+09,NVDA
2,2024-06-28,1136.21,1255.8700,117.0100,123.54,5.234031e+09,NVDA
3,2024-05-31,850.77,1158.1915,812.5456,1096.33,9.647971e+08,NVDA
4,2024-04-30,902.99,922.2500,756.0600,864.02,1.004974e+09,NVDA


In [43]:
def load_data(data):
    # Nombre de la tabla y esquema
    esquema = "alejandroramirezz25_coderhouse"
    tabla = "finance_nvidia"
    # Crea la conexión a Redshif
    print("Conectando a Redshift...")
    conn = redshift_connector.connect(
        host=os.getenv("REDSHIFT_HOST"),
        port=int(os.getenv("REDSHIFT_PORT")),
        database=os.getenv("REDSHIFT_DATABASE"),
        user=os.getenv("REDSHIFT_USER"),
        password=os.getenv("REDSHIFT_PASSWORD")
    )
    print('Conexion a Redshift creada...')
        # Crea la tabla en el esquema especificado
    create_table_query = f'''
        CREATE TABLE IF NOT EXISTS {esquema}.{tabla} (
            "date" DATE distkey,
            "open" VARCHAR(255),
            high VARCHAR(255),
            low VARCHAR(255),
            close VARCHAR(255), 
            volume VARCHAR(255),
            symbol VARCHAR(255)
        ) sortkey(date);
    '''

    with conn.cursor() as cursor:
        print(f"Creando tabla {tabla}")
        cursor.execute(create_table_query)
        print(f"Insertando datos...")
        for index, row in data.iterrows():
            cursor.execute(
                f'''INSERT INTO {esquema}.{tabla} ("date", "open", high, low, close, volume, symbol) VALUES (%s, %s, %s, %s, %s, %s, %s)''',
                (row['date'], row['open'], row['high'], row['low'], row['close'], row['volume'], row['symbol'])
            )
        cursor.execute(f"SELECT * FROM {esquema}.{tabla} LIMIT 10")
        results = cursor.fetchall()

        column_names = [desc[0] for desc in cursor.description]
        table = tabulate(results, headers=column_names, tablefmt='psql')
        print(table)
    conn.commit()

    # Cierra la conexión a Redshift
    conn.close()

In [44]:
load_data(transformed_data)

Conectando a Redshift...


TypeError: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'